In [1]:
from FeedbackGenerator import FeedbackGenerator
import tensorflow as tf
import numpy as np
import tensorflow.keras as k
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam

from tensorflow.python.client import device_lib

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

K.set_image_data_format('channels_first');

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:41:00.0, compute capability: 7.5



## Load Data

In [2]:
data = np.load(r"../TrainingData/feedback_data.npz")
x_train = data['x_train']
y_train = data['y_train']
print(x_train[0].shape,y_train[0].shape)
x_test = data['x_test']
y_test = data['y_test']
print(x_test.shape,y_test.shape)
#del data

(5, 8, 128, 128) (10, 8, 128, 128)
(276, 5, 8, 128, 128) (276, 10, 8, 128, 128)


## Visualizing results

In [3]:
import copy
from numpy.random import shuffle
import matplotlib.pyplot as plt
from PIL import Image

shuffle(viz := copy.deepcopy(x_test))
#viz = viz[0:5,:,:,:,:]
imnum=0
def visualize(model:k.Model):
  def save_gif(frames,name:str):
    if name[-4:] != '.gif':
      name += '.gif'
    frames[0].save(name,format='GIF',append_images=frames[1:],save_all=True,loop=0)
  print(viz.shape)
  try:
    preds = [model.predict(viz[i:i+1,...]) for i in range(5)]
  except Exception as e:
    print(e)
  #preds = K.get_value(preds)
  
  plt.figure(figsize=((4,10)))
  fig,ax = plt.subplots(2,5)
  for i in range(5):
    s=ax[0,i].imshow(np.squeeze(preds[i][0,0,4,:,:]),cmap='gray')
    s=ax[0,i].get_xaxis().set_visible(False)
    s=ax[0,i].get_yaxis().set_visible(False)  
    s=ax[1,i].imshow(np.squeeze(viz[i,0,4,:,:]),cmap='gray')
    s=ax[1,i].get_xaxis().set_visible(False)
    s=ax[1,i].get_yaxis().set_visible(False)  
  s=plt.show()
  for i in range(5):
    gif = [*[Image.fromarray(viz[i,j,4,:,:]) for j in range(5)],Image.fromarray(preds[i][0,0,4,:,:])]
    save_gif(gif, f'gif_{imnum}_{i}.gif')
  print('here')

class VisualizerCallback(k.callbacks.Callback):
  def __init__(self,data):
    super(VisualizerCallback, self).__init__()
    self.data = data[0:5,...]
  def on_epoch_end(self, batch, logs=None):
    preds = self.model(viz)
    preds = K.get_value(preds).astype(np.uint8)
    for i in range(5):
      gif = [*[Image.fromarray(viz[i,j,4,:,:]) for j in range(5)],Image.fromarray(preds[i,0,4,:,:])]
      save_gif(gif, f'gif_{imnum}_{i}.gif')
visualizer = VisualizerCallback(viz)

EarlyStop = k.callbacks.EarlyStopping(monitor='loss',patience=5, restore_best_weights=True)

## Make Model

In [7]:
from FeedbackGenerator import FeedbackGenerator
try: del ae
except: pass
ae = FeedbackGenerator(do_batch_norm=False,use_noise=True)
ae.compile(loss='huber', optimizer = Adam(learning_rate=.0001),run_eagerly=True);
ae.build([50,5,8,128,128]);
ae.summary()

Model: "SimpleU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv3D_0 (Conv3D)            multiple                  680       
_________________________________________________________________
Conv3D_1 (Conv3D)            multiple                  1360      
_________________________________________________________________
Conv3D_2 (Conv3D)            multiple                  4065      
_________________________________________________________________
Conv3D_3 (Conv3D)            multiple                  8120      
_________________________________________________________________
DownActivation_0 (PReLU)     multiple                  163840    
_________________________________________________________________
DownActivation_1 (PReLU)     multiple                  81920     
_________________________________________________________________
DownActivation_2 (PReLU)     multiple                  1536

ValueError: You tried to call `count_params` on Noise_1, but the layer isn't built. You can build it manually via: `Noise_1.build(batch_input_shape)`.

## Train UGEN

In [5]:
'''
for i in range(5):
  ae.fit(x_train,y_train,epochs=10,batch_size=50,shuffle=True)
  visualize(ae)
'''
ae.fit(x_train,y_train,epochs=100,batch_size=50,shuffle=True,callbacks=[EarlyStop])

Epoch 1/100
24/24 [==============================] - 69s 2s/step - loss: 0.0568
Epoch 2/100
24/24 [==============================] - 54s 2s/step - loss: 0.0431
Epoch 3/100
24/24 [==============================] - 55s 2s/step - loss: 0.0404
Epoch 4/100
24/24 [==============================] - 54s 2s/step - loss: 0.0324
Epoch 5/100
24/24 [==============================] - 54s 2s/step - loss: 0.0262
Epoch 6/100
24/24 [==============================] - 51s 2s/step - loss: 0.0235
Epoch 7/100
24/24 [==============================] - 51s 2s/step - loss: 0.0225
Epoch 8/100
18/24 [=====================>........] - ETA: 12s - loss: 0.0230

KeyboardInterrupt: 

## Looping Video

In [6]:
#shuffle(viz := copy.deepcopy(x_train))
vid = viz[1:2,...]
print(vid.shape)
for f in range(50):
  frame = ae.predict(vid[:,f:f+5,...])
  vid = np.append(vid,frame,axis=1)

def save_gif(frames,name:str):
    if name[-4:] != '.gif':
        name += '.gif'
    frames[0].save(name,format='GIF',append_images=frames[1:],save_all=True,loop=0)

save_gif([Image.fromarray(vid[0,j,4,:,:]*255) for j in range(15)],'prediction.gif')

save_gif([Image.fromarray(vid[0,j,4,:,:]*255) for j in range(vid.shape[1])],'super_prediction.gif')

(1, 5, 8, 128, 128)
